In [1]:
using Yao

# QFT circuit

In [ ]:
A(i, j) = control(i, j=>shift(2π/(1<<(i-j+1))))

In [7]:
R4 = A(4, 1) # qubit num can be desided after

(n -> control(n, 4, 1 => shift(0.39269908169872414)))

In [8]:
R4(5)

nqubits: 5
control(4)
└─ (1,) shift(0.39269908169872414)

In [13]:
B(n, k) = chain(n, j==k ? put(k=>H) : A(j, k) for j in k:n)
# if j==k, put(k=>H), else, A(j, k)~

B (generic function with 1 method)

In [14]:
qft(n) = chain(B(n, k) for k in 1:n)

qft (generic function with 1 method)

In [11]:
qft(4)

nqubits: 4
chain
├─ chain
│  ├─ put on (1)
│  │  └─ H
│  ├─ control(2)
│  │  └─ (1,) shift(1.5707963267948966)
│  ├─ control(3)
│  │  └─ (1,) shift(0.7853981633974483)
│  └─ control(4)
│     └─ (1,) shift(0.39269908169872414)
├─ chain
│  ├─ put on (2)
│  │  └─ H
│  ├─ control(3)
│  │  └─ (2,) shift(1.5707963267948966)
│  └─ control(4)
│     └─ (2,) shift(0.7853981633974483)
├─ chain
│  ├─ put on (3)
│  │  └─ H
│  └─ control(4)
│     └─ (3,) shift(1.5707963267948966)
└─ chain
   └─ put on (4)
      └─ H


### Wrap QFT to an external block

it's useful to be able to wrap circuit to custom blocks.

In [88]:
# Parametric Composite Types
struct QFT{N} <: PrimitiveBlock{N} end # define new type QFT{N} as subtype of PrimitiveBlock
QFT(n::Int) = QFT{n}() # QFT{n::int} class is defined.

QFT

In [148]:
println( typeof(QFT) )
println( typeof(QFT(3)) )

UnionAll
QFT{3}


In [127]:
circuit(::QFT{N}) where N = qft(N)
# circuit(::QFT{N}) where N  *ここが区切り*  = qft(N)

circuit (generic function with 2 methods)

In [145]:
circuit(QFT(3))

nqubits: 3
chain
├─ chain
│  ├─ put on (1)
│  │  └─ H
│  ├─ control(2)
│  │  └─ (1,) shift(1.5707963267948966)
│  └─ control(3)
│     └─ (1,) shift(0.7853981633974483)
├─ chain
│  ├─ put on (2)
│  │  └─ H
│  └─ control(3)
│     └─ (2,) shift(1.5707963267948966)
└─ chain
   └─ put on (3)
      └─ H


In [142]:
println( typeof(QFT(2)) )
println( typeof(circuit(QFT(2))) )
println( typeof(qft(2)) )

QFT{2}
ChainBlock{2}
ChainBlock{2}


In [132]:
circuit(QFT(3)) == qft(3)

true

In [38]:
mat(circuit(QFT(2)))

4×4 SparseArrays.SparseMatrixCSC{Complex{Float64},Int64} with 16 stored entries:
  [1, 1]  =  0.5+0.0im
  [2, 1]  =  0.5+0.0im
  [3, 1]  =  0.5+0.0im
  [4, 1]  =  0.5+0.0im
  [1, 2]  =  0.5+0.0im
  [2, 2]  =  -0.5+0.0im
  [3, 2]  =  0.5+0.0im
  [4, 2]  =  -0.5+0.0im
  [1, 3]  =  0.5+0.0im
  [2, 3]  =  3.06162e-17+0.5im
  [3, 3]  =  -0.5+0.0im
  [4, 3]  =  -3.06162e-17-0.5im
  [1, 4]  =  0.5+0.0im
  [2, 4]  =  -3.06162e-17-0.5im
  [3, 4]  =  -0.5+0.0im
  [4, 4]  =  3.06162e-17+0.5im

In [79]:
YaoBlocks.mat(::Type{T}, x::QFT) where T = mat(T, circuit(x))
# YaoBlocks の　method(generic function) を今回作った型で定義している??

In [41]:
YaoBlocks.print_block(io::IO, x::QFT{N}) where N = print(io, "QFT($N)")

Since it is possible to use FFT to simulate the results of QFT (like cheating), we could define our custom apply! method:

In [51]:
using FFTW, LinearAlgebra

function YaoBlocks.apply!(r::ArrayReg, x::QFT)
    α = sqrt(length(statevec(r)))
    invorder!(r)
    lmul!(α, ifft!(statevec(r)))
    return r
end

In [48]:
r = rand_state(5)
r1 = r |> copy |> QFT(5)
r2 = r |> copy |> circuit(QFT(5))
r1 ≈ r2

true

In [67]:
r1 = r |> copy |> QFT(5) |> r -> measure(r, nshots=100)

100-element Array{BitBasis.BitStr{5,Int64},1}:
 00000 ₍₂₎
 11001 ₍₂₎
 11001 ₍₂₎
 10000 ₍₂₎
 11001 ₍₂₎
 01111 ₍₂₎
 01111 ₍₂₎
 10011 ₍₂₎
 11100 ₍₂₎
 10111 ₍₂₎
 01010 ₍₂₎
 10010 ₍₂₎
 10010 ₍₂₎
         ⋮
 01011 ₍₂₎
 01111 ₍₂₎
 11010 ₍₂₎
 00111 ₍₂₎
 00001 ₍₂₎
 00111 ₍₂₎
 01000 ₍₂₎
 10000 ₍₂₎
 01110 ₍₂₎
 10010 ₍₂₎
 11000 ₍₂₎
 10010 ₍₂₎

In [151]:
QFT(5)'

 [†]QFT(5)

In [159]:
println(typeof(QFT(5)'))
println(typeof(qft(3)'))

Daggered{QFT{5},5}
ChainBlock{3}


# Phase Estimation circuit

In [152]:
Hadamards(n) = repeat(H, 1:n)

Hadamards (generic function with 1 method)

In [153]:
ControlU(n, m, U) = chain(n+m, control(k, n+1:n+m=>matblock(U^(2^(k-1)))) for k in 1:n)

ControlU (generic function with 1 method)

In [154]:
PE(n, m, U) =
    chain(n+m, # total number of the qubits
        concentrate(Hadamards(n), 1:n), # apply H in local scope
        ControlU(n, m, U),
        concentrate(QFT(n)', 1:n))

PE (generic function with 1 method)

一度CahinBlock{N}として定義したqft(n)を、QFT(n)という名前でPrimitiveBlock{N}として定義し直したため、もう一度chain()に突っ込める。

In [155]:
r = rand_state(5)

ArrayReg{1, Complex{Float64}, Array...}
    active qubits: 5/5

In [156]:
focus!(r, 1:3)

ArrayReg{1, Complex{Float64}, Array...}
    active qubits: 3/5

In [157]:
relax!(r, 1:3)

ArrayReg{1, Complex{Float64}, Array...}
    active qubits: 5/5

In this way, we will be able to apply small operator directly on the subset of the qubits.

In [161]:
N, M = 3, 5
P = eigen(rand_unitary(1<<M)).vectors
θ = Int(0b110) / 1<<N
phases = rand(1<<M)
phases[0b010+1] = θ
U = P * Diagonal(exp.(2π * im * phases)) * P'

32×32 Array{Complex{Float64},2}:
     0.46071-0.302347im    …   -0.0903844-0.101259im
   0.0690542-0.1068im          -0.0584892-0.0241442im
   0.0376043+0.0355771im       -0.0954238-0.150688im
   0.0617421-0.12045im        -0.00245556-0.22158im
   0.0321338+0.0142511im       -0.0605465-0.151011im
   0.0203351+0.114761im    …    -0.138323-0.0715385im
   0.0671265+0.065407im          0.143348-0.100983im
   0.0228185-0.0983436im        -0.101941+0.0920513im
   0.0362004-0.027697im        0.00568236-0.207122im
 -0.00114995+0.0190755im        0.0150346+0.0654877im
  -0.0426691-0.0456888im   …   -0.0752605+0.152529im
    0.155556+0.0184419im        0.0121835+0.156086im
   0.0780916-0.0349016im        0.0725806+0.113856im
            ⋮              ⋱  
     0.20813-0.0671839im   …     0.229504-0.124093im
  -0.0561755+0.0910277im       0.00887671-0.13519im
   0.0489875-0.241589im         0.0891129-0.0522965im
   -0.056753-0.116786im         -0.135559-0.000524529im
    0.143653+0.0172643im     

In [162]:
psi = P[:, 3]

32-element Array{Complex{Float64},1}:
  -0.05849257886632364 - 0.17791781646085325im
   0.06013684973829846 + 0.05935300087228702im
   0.20740409946452612 - 0.18818220270461297im
   0.12139343030157221 + 0.056267662670782384im
  0.007786324597054674 - 0.22020470546794624im
   0.17529926509884827 - 0.1445920624896402im
  -0.11954995064266083 - 0.15730177195456566im
   0.13605363808283408 - 0.013756616230986222im
  -0.22340332394751417 + 0.10236410312442945im
  -0.10477667654391569 - 0.07089751093575208im
  -0.19535410346623142 + 0.10134837875973857im
   0.12758298333611373 - 0.07137772153111205im
  -0.11913736299928815 + 0.03476389104599038im
                       ⋮
  -0.04660587058350546 - 0.08309812553125542im
  -0.08781714059506278 - 0.058886343995567636im
   0.23720806565730151 + 0.17213190799575814im
  -0.09338030808645637 - 0.15733585308739914im
  -0.16010477524858918 - 0.1481030303440515im
   0.03914536533728363 - 0.09949153546566986im
  0.043028930837625434 + 0.1557243845375818

In [163]:
r = join(ArrayReg(psi), zero_state(N))
r |> PE(N, M, U)

┌ Warning: `concentrate` is deprecated, use `subroutine` instead.
│   caller = PE(::Int64, ::Int64, ::Array{Complex{Float64},2}) at In[154]:1
└ @ Main ./In[154]:1
┌ Warning: `concentrate` is deprecated, use `subroutine` instead.
│   caller = PE(::Int64, ::Int64, ::Array{Complex{Float64},2}) at In[154]:1
└ @ Main ./In[154]:1


ArrayReg{1, Complex{Float64}, Array...}
    active qubits: 8/8

In [164]:
results = measure(r, 1:N; nshots=1)

1-element Array{BitBasis.BitStr{3,Int64},1}:
 011 ₍₂₎

In [167]:
using BitBasis
estimated_phase = bfloat(results[]; nbits=N)

0.75